In [42]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

BASE = "https://encuestas-finales.exactas.uba.ar/"
INICIO = BASE + "periodos.html"

soup = bs(requests.get(INICIO).text, "html.parser")

ordinales = {"1": "Primer", "2": "Segundo", "3": "Tercero", "4":"Cuarto"}
epocas = {"i": "Invierno", "v": "Verano", "c": "Cuatrimestre", "b": "Bimestre"}

def lst(lid, p):
    print("LST", lid, p)
    url = BASE + "lists/l_"  + lid + "_" + p + ".html"
    return requests.get(url).text


def parsear_periodo(periodo):
    if len(periodo) == 5:
        if periodo[0] == "a":
            return

        epoca = epocas[periodo[0]]
        año = periodo[1:]
    
    elif len(periodo) == 6:
        epoca = ordinales[periodo[0]] + " " + epocas[periodo[1]]
        año = periodo[2:]

    else:
        print("Error en el formato del periodo", periodo)
        return

    return epoca, año
    # data = []
    # soup = bs(requests.get(url).text, "html.parser")
    # paginas = soup.find_all("div", class_="list")[0]
    # id_ = paginas.attrs["id"].split("_")[-1]
    # links = paginas.find_all("div", class_="head")[0].find_all("a")
    # for link in links:
    #     p = link.text
    #     lst(id_, p)
    

# encuestas = []
# for fila in soup.find_all("tr"):
#     if fila.find("th"):
#         continue
#     periodo = fila.find_all("td")[0].text
#     url = BASE + fila.find_all("td")[1].find("a")["href"]
#     if periodo == "2c2022":
#         parsear_periodo(periodo, url)

In [58]:
def parsear_materias():
    PAGINAS = 74
    materias = {}
    for i in range(PAGINAS):
        html = lst("mats", str(i))
        for materia in bs(html, "html.parser").find("ul").find_all("li"):
            # Save materia withthetext as a key and the link as a value
            link = materia.find("a")
            materias[link.text] = BASE + link["href"]

    import json
    with open("materias.json", "w") as f:
        json.dump(materias, f)

parsear_materias()

LST mats 0
LST mats 1
LST mats 2
LST mats 3
LST mats 4
LST mats 5
LST mats 6
LST mats 7
LST mats 8
LST mats 9
LST mats 10
LST mats 11
LST mats 12
LST mats 13
LST mats 14
LST mats 15
LST mats 16
LST mats 17
LST mats 18
LST mats 19
LST mats 20
LST mats 21
LST mats 22
LST mats 23
LST mats 24
LST mats 25
LST mats 26
LST mats 27
LST mats 28
LST mats 29
LST mats 30
LST mats 31
LST mats 32
LST mats 33
LST mats 34
LST mats 35
LST mats 36
LST mats 37
LST mats 38
LST mats 39
LST mats 40
LST mats 41
LST mats 42
LST mats 43
LST mats 44
LST mats 45
LST mats 46
LST mats 47
LST mats 48
LST mats 49
LST mats 50
LST mats 51
LST mats 52
LST mats 53
LST mats 54
LST mats 55
LST mats 56
LST mats 57
LST mats 58
LST mats 59
LST mats 60
LST mats 61
LST mats 62
LST mats 63
LST mats 64
LST mats 65
LST mats 66
LST mats 67
LST mats 68
LST mats 69
LST mats 70
LST mats 71
LST mats 72
LST mats 73


In [105]:
import json
import cv2
import hashlib
materias = json.load(open("materias.json"))

respuestas_hash = {}

def parsear_periodo(periodo):
    if len(periodo) == 5:
        if periodo[0] == "a":
            return None, None

        epoca = epocas[periodo[0]]
        año = periodo[1:]
    
    elif len(periodo) == 6:
        epoca = ordinales[periodo[0]] + " " + epocas[periodo[1]]
        año = periodo[2:]

    else:
        print("Error en el formato del periodo", periodo)
        return None, None

    return epoca, año

def parsear_respuestas(url):
    global respuestas_hash
    PREGUNTAS = 17
    imagen = requests.get(url).content
    with open("/tmp/respuestas_exactas.png", "wb") as f:
        f.write(imagen)

    img = cv2.imread("/tmp/respuestas_exactas.png")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY)[1]

    width = img.shape[1]
    width_res = width // PREGUNTAS + 1

    respuestas = {}
    for i in range(PREGUNTAS-1):
        respuesta = img[:,width_res*i:width_res*(i+1)-1]
        hasheada = hashlib.md5(respuesta.data.tobytes()).hexdigest()
        if not hasheada in respuestas_hash:
            respuestas_hash[hasheada] = respuesta
            cv2.imwrite(f"respuestas/{hasheada}.png", respuesta)   

        respuestas[f"p{i}"] = hasheada
    
    return respuestas

def parsear_comentarios(id_materia, id_cuatri):
    url = BASE + "cma/" + id_materia + id_cuatri + ".html"
    soup = bs(requests.get(url).text, "html.parser")
    
    comentarios = []
    for comentario in soup.find_all("div", class_="cm"):
        comentarios.append(comentario.text.replace('\r', '\n'))

    return comentarios

def parsear_materia(materia, url):
    soup = bs(requests.get(url).text, "html.parser")

    id_materia = url.split("/")[-1].split(".")[0]
    
    departamento = None
    encuestas = []
    
    for bold in soup.find_all("b"):
        if bold.text == "Departamento:":
            departamento = bold.next_sibling.next_sibling.text
    
    # Find tr elements with non empty ids and withoout  display none
    for fila in soup.find_all("tr"):
        if not fila.has_attr("id"):
            continue
        if fila.has_attr("style"):
            continue

        id_cuatri = fila.attrs["id"]
        periodo = fila.find_all("td")[0].text
        periodo = parsear_periodo(periodo)
        foto_url = '/'.join(fila.find("img")["src"].split("/")[1:])
        print(materia, periodo)

        respuestas = parsear_respuestas(BASE + foto_url)
        comentarios = parsear_comentarios(id_materia, id_cuatri)

        encuestas.append({
            "materia": materia,
            "departamento": departamento,
            "epoca": periodo[0],
            "año": periodo[1],
            "comentarios": comentarios,
            **respuestas
        })
    
    return encuestas
        

encuestas = []
for materia, url in materias.items():
    encuestas +=  parsear_materia(materia, url)

import pandas as pd
df = pd.DataFrame(encuestas)
df.to_csv("encuestas.csv")
df.head()

Analisis de Datos ('Primer Cuatrimestre', '2008')
Analisis de Datos ('Primer Cuatrimestre', '2006')
Analisis de Datos ('Segundo Cuatrimestre', '2003')
Acoplamiento y Procesos Estocasticos ('Segundo Cuatrimestre', '2004')
Acuaporinas:estructura y Funcion ('Segundo Cuatrimestre', '2012')
Administracion de Proyectos Informaticos ('Segundo Cuatrimestre', '2011')
Administracion de Proyectos Informaticos ('Segundo Cuatrimestre', '2010')
Administracion de Proyectos Informaticos ('Primer Cuatrimestre', '2006')
Administracion de Proyectos Informaticos ('Segundo Cuatrimestre', '2005')
Administracion de Proyectos Informaticos ('Primer Cuatrimestre', '2005')
Administracion de Proyectos Informaticos ('Segundo Cuatrimestre', '2004')
Administracion de Proyectos Informaticos ('Primer Cuatrimestre', '2004')
Administracion de Proyectos Informaticos ('Segundo Cuatrimestre', '2003')
Administracion de Proyectos Informaticos ('Primer Cuatrimestre', '2003')
Administracion de Proyectos Informaticos ('Segundo 

TypeError: 'NoneType' object is not subscriptable